### same as the article on the website, but using averagepooling instead of max pooling

- this is roughly based upon https://github.com/fchollet/keras/blob/master/examples/imdb_fasttext.py

In [13]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, Dropout, Embedding, GlobalMaxPooling1D, GlobalAvgPool1D
from keras.models import Model,Sequential

from helpers import files

In [2]:
embeddings_index = files.read_glove_100()

In [3]:
texts,labels_index,labels_0 = files.read_20_newgroup_files()       

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

In [5]:
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_2[-num_validation_samples:]

In [6]:
len(word_index)

174074

In [7]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = True))

model.add(GlobalAvgPool1D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(labels_index), activation='softmax'))

model.compile(loss ='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [24]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=10, batch_size=32)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 24s - loss: 2.8545 - acc: 0.1209 - val_loss: 2.6089 - val_acc: 0.2348
Epoch 2/10
15998/15998 [==============================] - 24s - loss: 2.4161 - acc: 0.2388 - val_loss: 2.1537 - val_acc: 0.4066
Epoch 3/10
15998/15998 [==============================] - 24s - loss: 2.0637 - acc: 0.3219 - val_loss: 1.8496 - val_acc: 0.4954
Epoch 4/10
15998/15998 [==============================] - 24s - loss: 1.7998 - acc: 0.4088 - val_loss: 1.6346 - val_acc: 0.5511
Epoch 5/10
15998/15998 [==============================] - 24s - loss: 1.5886 - acc: 0.4861 - val_loss: 1.4568 - val_acc: 0.5904
Epoch 6/10
15998/15998 [==============================] - 24s - loss: 1.4140 - acc: 0.5485 - val_loss: 1.3113 - val_acc: 0.6422
Epoch 7/10
15998/15998 [==============================] - 24s - loss: 1.2614 - acc: 0.5983 - val_loss: 1.2089 - val_acc: 0.6404
Epoch 8/10
15998/15998 [==============================]